# Example Validation Template for a client

In [219]:
## Setting up environment

# importing generic packages 
from pyathena import connect
import pandas as pd
import io
import numpy as np 
import warnings
import boto3
from datetime import datetime, timedelta, date
# import bpm data validation package
import dash_validation_toolkit 
# stop warnings
import warnings
from datetime import datetime, timedelta, date

warnings.filterwarnings('ignore')
# create athena connection to uk data hub
athena_conn = connect(s3_staging_dir='s3://bucket_path/',
               region_name='region')

# Define date variables
from datetime import datetime, timedelta, date

# Add current week 
today = datetime.today()
this_monday_date = today - timedelta(days = today.weekday()
                           , hours = today.hour
                           , minutes = today.minute
                           , seconds = today.second
                           , microseconds = today.microsecond
                          )
# convert to string
this_monday = this_monday_date.strftime('%Y%m%d')

# getting last monday date
last_monday = this_monday_date - timedelta(days = 7)
last_monday = last_monday.strftime('%Y%m%d')

## Client set-up

In [220]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
# the string name of the client to be validated
validation_client = 'example'
# a list of the aws regions the client operates in 
regions = ["ap-southeast-1", "us-east-1"]
# a list of the prod core databases for the client in their origin region
core_databases = ["core data base"]
# a list of the prod core databases for the client in their origin region
zephyr_databases = ["zephyr image database"]
# Defining the webhook for sending messages to slack:
webhook = "webhook api address"


In [221]:
# creating the clients object for adding warnings and errors to throughout the validation stages:
client_object = client(
                       client=validation_client, 
                       regions=regions,
                       core_databases=core_databases, 
                       zephyr_databases=zephyr_databases, 
                       slack_webhook=webhook, 
                       connection=athena_conn, 
                       send_to_slack=True
                      )

## Stage 1 Checks (Regional Prod to Union Comparisons) 

In [222]:
# Stage 1: Validation of Production Union tables: Loop through all union tables of a client & ensure all pass stage 1 validations, with any failures or warnings passing to dependency tables.
#      i.   1.1 - Check count of each region in prod is correctly represented in aggregated union table 
#      ii.  1.2 - Check that there are no nulls in region columns of union table - null check

### Run all Stage 1 checks

<b> Set up: Create check_set </b> <br>
Declare which checks to run for this stage - only checks added as strings below in check_set will be run. 
> Note: if wanting to skip stage entirely than leave as an empty set

In [223]:
check_set = {'1.1', '1.2'}

In [ ]:
# Running the stage one checks, for the client using stage one driver function, with no user-defined client inputs required :
stage_1_driver = stage_1(name='Stage 1', client=client_object, check_set=check_set).run_stage_checks()

Check 1.1 (same regional row count):  True 85 menicon_order_configurations jj_prod_union 85 menicon_order_configurations core_apse1_prod
Check 1.1 (same regional row count):  True 360 patients jj_prod_union 360 patients core_apse1_prod
Check 1.1 (same regional row count):  True 355 menicon_lens_parameters jj_prod_union 355 menicon_lens_parameters core_apse1_prod
Check 1.1 (same regional row count):  True 203 organisational_unit_user jj_prod_union 203 organisational_unit_user core_apse1_prod
Check 1.1 (same regional row count):  True 113 addresses jj_prod_union 113 addresses core_apse1_prod
Check 1.1 (same regional row count):  True 355 menicon_order_items jj_prod_union 355 menicon_order_items core_apse1_prod
Check 1.1 (same regional row count):  True 10 ethnicities jj_prod_union 10 ethnicities core_apse1_prod
Check 1.1 (same regional row count):  True 0 menicon_ended_reasons jj_prod_union 0 menicon_ended_reasons core_apse1_prod
Check 1.1 (same regional row count):  True 251 custom_addr

## Stage 2 Checks (Base & FHIR table validation)

In [ ]:
    # Stage 2: Validation of Base and FHIR tables: Loop through all base and FHIR tables for client, ensuring all pass stage 2 validations, with any prior failures/warnings or new failure/warnings carrying across into dependant tables.
    #      i.   2.1 - Check that primary key count between union and base/FHIR tables is the same
    #      ii.  2.2 - Check that primary key of base/FHIR are all in primary key of union table - distinct primary keys are the same
    #      iii. 2.3 - Check that every primary key is unique within base/FHIR tables - for base/FHIR tables this is only the primary id and not the update time
    #      iv. 2.4 - Check that defintion look-up tables are up-to-date & not missing new values for FHIR & Base tables
    #       v. 2.5 - Check that a particular tracked metric is the expected value - e.g. no. of open consults or missing postcodes

In [ ]:
# Set-up stage 2 parameter Dictionary, containing all check lists for stage - and each list containing all check instances (with custom parameters) to be completed :
stage_parameters = { # list for checks 2.1, 2.2, 2.3
                     "source_target_input_list"   : list(),
                     # list for check 2.4
                     "definition_check_list": list(),
                     # list for check 2.5:
                     "track_check_list" : list()
                    } 

### Set-up checks 2.1, 2.2, 2.3

<b> Set up: Create query list </b> <br>
For primary key checks. 2.1-2.3 - this will check that primary key count is same between union and base/fhir tables, check the primary key constraint for each table and that all primary keys are present between both union and base/fhir tables. The input parameters are as follows:

> target: the name of the table being tested 

> target_PK:  an integer representing the index number of the PK for the base/FHIR table e.g. in format 0 (index 0 of base table) - or if composite base/FHIR PK key then a list of column indexes e.g. dim_user PK would be [0,7]

> target_PK: the corresponding union query from the 'FROM' downwards only (not select) which would give the PK id equivalent for the corresponding union or source table to compare against  e.g. in the format: """ FROM jj_prod_union.menicon_encounters """]

> parent_query: a string representing the name of the corresponding PK column of the union or Source table e.g. in format 'id' OR 'id, region' if composite PK key

> prod_ids: a string of the target database table is in - e.g. sandbox or base_tables

> targetdbs: a bool True or False to signify whether to complete the unique PK check or not - e.g. for cases where we want to allow breaking of constraints/checks

#### (i) Base tables query list

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
stage_parameters["source_target_input_list"].append( 
{   "target": "TABLE_name",
    "target_PK": 'encounter_id',
    "parent_query": """  FROM   jj_prod_union.menicon_encounters   """,
    "prod_ids": 'id',
    "targetdbs": "jj_sandbox" ,
})


#### (ii) FHIR tables query list

In [ ]:

#Dim tables against production/union
stage_parameters["source_target_input_list"].append(
{   "target": "TABLE_name",
    "target_PK": "column_id",
    "parent_query":  "FROM   TABLE_name.column_id", 
    "prod_ids": 'id', 
    "targetdbs": "DATABASE" 
})

### Set-up check 2.4

<b> Create the look-up table list </b><br>
For checking no new/uncategorised defintion values in client related look-up tables, and then the parameters below for the check are as follows:

> definition: the name of the defintion e.g. ecp role - which will be used to display name of definiton being checked in the output

> look_up_database: the string name of the database for which the look-up table is located e.g. jj_sandbox

> look_up_table:  the name of the look-up table e.g. in the format: 'ecp_user_roles_LOOK_UP' for JJ (table with the look_up_inclusion_flag for the definition value) 

> look_up_column: a string with the name of column representing the definition value being monitored - e.g. name in the case of ecp roles

> look_up_inclusion_flag:  a string, look-up inclusion flag, which indicates the column used for categorising the look-up value - if none then look-up value column of interest - e.g. name for cost roles or inclusion_flag for ecp_roles

> Note: look_up_column and look_up_inclusion_flag should not be the same column, if so consider using the track check with a value set output instead


In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################

# Definition 1: Creating check input variables for check 2.4, 
definition = "definition name"
stage_parameters["definition_check_list"].append(
{   "definition": definition,
    "look_up_database": 'DATABASE',
    "look_up_table":  "LOOK_UP_COLUMN", 
    "look_up_column": 'COLUMN', 
    "look_up_inclusion_flag": "look_up_inclusion_flag" 
})


### Set-up check 2.5

<b> Check for a tracked metric that the it is tracking at its expected value </b> <br>
For checking that a tracked metric (e.g. orgs with null countries) has the expected/desired number (e.g. 0) or returns any values from query if a set of values
Parameters are as follows:

> tracking_name: name of the tracking check

> tracking_query: the query string for getting the current count of the tracked thing (e.g. orgs with null country) - need to name either count or set depending on desired query output

> expected_result: an integer for what the expected or desired number of this tracked thing should be

> track_type: a string indicating whether query output is a count to be compared against number OR a set of values to be shown if not an empty set returned


In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
                                                                            
# setting up input for tracking statistic check
tracking_name = "name of tracking stat"
stage_parameters["track_check_list"].append(
{   "tracking_name": tracking_name,
    "tracking_query":  """
                        SELECT DISTINCT COLUMN_OF_INTEREST AS set
                        FROM "DATABASE"."TABLE" 
                        
                       """,
    "expected_result":  0, 
    "track_type": 'set' 
})


### Run all Stage 2 Checks

<b> Set up: Create check_set </b> <br>
Declare which checks to run for this stage - only checks added as strings below in check_set will be run. 
> Note: if wanting to skip stage entirely than leave as an empty set

In [ ]:
check_set = {'2.1', '2.2', '2.3', '2.4', '2.5'}

In [ ]:
# Running the stage two checks, for the client using stage one driver function, with no user-defined client inputs required :
client_object = stage_2(name='Stage 2', client=client_object, check_set=check_set, **stage_parameters).run_stage_checks()

## Stage 3 Checks (Base to Dashboard Comparisons)

In [ ]:
    # Stage 3: Validation of Dashboard tables: For each client, check end-point powerBI dashboard tables, comparing key dashboard statistics against base, checking assumptions & checking business logic
    #      i.   3.1 - Check that if a statistic has a select all (except onboarded) that all pathways add up to select all
    #      ii.  3.2 - Check that dashboard table statistics count against associated base table is equal
    #      iii. 3.3 - Check a cumulative table, if select all than this is equal the overall base statistic total
    #      iii. 3.4 - Check for a statistic where all pathways should be same value (e.g. onboarded) that all pathways are equal
    #      v.  3.5 - Check for business logic against base tables
    #      iv.  3.6 - Within-dashboard comparisons, checking figures in dashboard are the same where they are meant to be

In [ ]:
# Set-up stage 3 parameter list, containing all check dictionary instances for stage - each check dictionary containing all custom parameters for a check to be completed
stage_parameters = {  # list for checks 3.1, 3.2 custom parameters
                     "between_dash_comparison_list" : list(),
                     # list for check 3.3 custom parameters
                     "dash_to_base_query_list": list(),
                     # list for check 3.4 custom parameters
                     "cumulative_check_list" : list(),
                     # list for checks 3.5 custom parameters
                     "onboard_stat_list"   : list(), 
                     # list for check 3.6 custom parameters
                     "business_logic_list": list() } 

### Set-up check 3.1, 3.2

<b> Create the query list for comparing base statistics to dashboard statistics </b> <br>


For checking dashboard statistic (non-onboard or cumulative stats - e.g. newly live ecp users or patients) has the same overall sum as the corresponding query for it off base tables. Input parameters required are:
> "dashboard_statistic":   the dashboard statistic name being checked

> "dash_table" a string with the name of the dashboard table - assumed to be in the client_dashboard tables database

> "base_statistic_query" the corresponding base table query for getting the overall sum of the base statisic (e.g. newly live ecp users off fact_users/practitioners)

>  "base_PK_by_pathways":  If the unique id/entity the statistic is based on for the client -e.g. patients - could be over more than 1 pathway. E.g. a patient on bloom day and bloom night, then
      this will result of double-counting of patients when pathway sums added together to compare against select all total. So if this is the case, then need to provide
      a query string for getting unique PK ids for statistic (eg patient id) for each pathway. If this is NOT the case, then just provide an empty list

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
dashboard_table = "dashboard_table_name"

# For each statistic that will be tested for sum base <-> dash check, will add logic to base_query_dictionary for obtaining the check against base  
# Adding logic for testing active ECP users in dashboard table against base table:
stage_parameters["dash_to_base_query_list"].append(
{
    "dashboard_statistic": "name of statistic",
    "dash_table": dashboard_table,
    "base_statistic_query":     """
                            SELECT COUNT(DISTINCT column_id) 
                            FROM BASE_DATABASE.base_table  u 
                                """, 
    "base_PK_by_pathways": []
})


### Set-up check 3.3

<b> Create the cumulative check list </b> </br>

For checking cumulative statistics, ensuring final week/month sum is equal to corresponding base table query sum and additionally select all is also equal. Input parameters are:

> i: name of the cumulative statistic in dash table

> i. a query string giving the cumulative stat summed grouped by level required for comparison - e.g. by country and pathway - taking max date - e.g. latest total cumulative patients

> ii. the summed statistic from base tables - e.g total patients

> iii. dictionary key being the name of the dashboard statistic being checked

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
# setting the cumulative dashboard query to be run for check 3.3 for ecp cumulative
cumulative_dash_query = """
                            -- take the latest cumulative stat for each country then sum up->should be equal to the overall base stat
                            SELECT 
                                SUM(dash_column_name) AS count
                            FROM 
                            DATABASE.TABLE
                        """
base_dash_query =   """
                        SELECT COUNT(DISTINCT base_id) AS count 
                        FROM DATABASE_base.TABLE_base
                    """

stage_parameters["cumulative_check_list"].append(
{
    "cumulative_dash_statistic": "name of dash column",
    "cumulative_dash_query": cumulative_dash_query,
    "base_dash_query": base_dash_query
})


### Set-up check 3.4

<b> Create the onboard statistic list  </b> </br>
For checking that for 'onboard' statistics where all totals should be the same over pathways and total/select all - e.g. onboarded users - that all levels are the same value, and in addition these are also equal to the corresponding query on the base tables - onboarded users. Input parameters are:

> "dashboard_statistic": name of onboard statistic in dash table

>  "base_statistic_query": a query string for the statistic off the base tables

> "dash_table": the name of the dashboard table

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
dashboard_table = "dash database"

# set-up for newly onboard ecps stat
base_dash_query =   """
                        SELECT COUNT(DISTINCT column_id) 
                        FROM database.table
                    """

stage_parameters["name of dash statistic"].append(
{
    "dashboard_statistic": "name of dash statistic",
    ""
    "dash_table": dashboard_table,
    "base_statistic_query": base_dash_query
})


### Set-up check 3.5

<b> Create the business logic list  </b> </br>
For checking that for 'onboard' statistics where all totals should be the same over pathways and total/select all - e.g. onboarded users - that all levels are the same value, and in addition these are also equal to the corresponding query on the base tables - onboarded users. Input parameters are:
 
> "business_logic_name": name you want to give to the business logic test - arbitary

> "business_logic_query" : a query string for the statistic off the base tables

> "base_stat_query":  the name of the dashboard onboard statiistic as the dictionary key 

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
# Checking total orders off base table is equal to logic off dashboard
business_logic_query = """ 
                            SELECT 
                            (
                                ( SUM(column1) + SUM(column2) + SUM(column3) ) 
                                + 
                                ( SUM(column4)  - SUM(column5) )  -

                            )   FROM database.table
                            """
base_stat_query =   """
                        SELECT COUNT(DISTINCT column_id) 
                        FROM database.table
                    """
stage_parameters["business_logic_list"].append(
{
    "business_logic_name": "column 1 + 2 +3 should == column 4 and 5 in dashboard",
    "business_logic_query": business_logic_query,
    "base_stat_query": base_stat_query
})


### Set-up check 3.6

<b> Create the between dash comparison list </b> <br>
 For checking that stats within the dashboard agree where they should e.g. total patients equals cumulative sum of select all for patients. YThough these checks will just check that output of two queries is the same, so can be used for other purposes aswell. Input parameters are:

> "dash_test_name": the name of the dashboard test - arbitary

> "dash_query_1": a query string for comparing one dashboard statistic query

>  "dash_query_2" : the second query string for comparing another dashboard statistic

>  "logical_comparion_operator": logical operator for wqhat comparsion between two outputs will be completed.

In [ ]:
#######################################################  REQUIRES INPUT FROM ANALYST ######################################################################
# Dashboard comparison check 1.  check select all of onboarded ecps is equal to the last date for the cumulative ecps for select all
# getting ecps count
dash_test_name = "test name"
dashboard_table = "dash database"
dash_query_1 =  """  	
                    SELECT SUM(dash_column1)
                    FROM dash_dbs.dash_tbl
                """
# getting ecps count from cumulative
dash_query_2 =  """
                    SELECT SUM(dash_column2) 
                    FROM FROM dash_dbs.dash_tbl
                    
                """
logical_operator =  "=="
stage_parameters["between_dash_comparison_list"].append(
{
    "dash_test_name": dash_test_name,
    "dash_query_1": dash_query_1,
    "dash_query_2": dash_query_2,
    "logical_comparion_operator": logical_operator
})

### Run all Stage 3 Checks 

<b> Set up: Create check_set </b> <br>
Declare which checks to run for this stage - only checks added as strings below in check_set will be run. 
> Note: if wanting to skip stage entirely than leave as an empty set

In [ ]:
check_set = {'3.1', '3.2', '3.3', '3.4', '3.5', '3.6'} 

In [ ]:
# Running the stage three checks, for the client using stage one driver function, with no user-defined client inputs required :
client_object = stage_3(name='Stage 3', client=client_object, check_set=check_set, **stage_parameters).run_stage_checks()

## Validation Output

> will output client validation results and output into slack

In [ ]:
# For the client, print out the warnings and errors of the validation and send to slack:
client_object.output_failures()